In [ ]:
from transformer_lens import HookedTransformer, HookedTransformerKeyValueCache, HookedTransformerKeyValueCacheEntry
import torch

batch_size = 8

model = HookedTransformer.from_pretrained_no_processing(
    model_name="Qwen/Qwen1.5-4B-Chat",
    device="cuda",
    dtype=torch.bfloat16,
    default_padding_side="left",
)
cache = HookedTransformerKeyValueCache.init_cache(model.cfg, model.cfg.device, batch_size)

logits = model(torch.randint(1, 1000, (batch_size, 32)), past_kv_cache=cache)

generating = [i for i in range(batch_size) if (i % 2)]

In [ ]:
print(len(cache.entries))
print(model.cfg.n_layers)

In [ ]:
entries = [
    HookedTransformerKeyValueCacheEntry(
        cache[i].past_keys[generating],
        cache[i].past_values[generating],
        cache[i].frozen,
    )
    for i in range(model.cfg.n_layers)
]
generating_cache = HookedTransformerKeyValueCache(entries, cache.previous_attention_mask[generating, :], cache.frozen)

In [ ]:
print(len(generating_cache.entries))
print(generating_cache.previous_attention_mask.shape)